In [1]:
import pandas as pd
import math
import os
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from IPython.display import display
import pandas as pd
import CGRtools
from CGRtools.files import SMILESRead, SDFWrite
from CGRtools.containers import MoleculeContainer, QueryContainer, QueryCGRContainer, ReactionContainer
from CGRtools.containers.bonds import DynamicBond
from CGRtools.periodictable import ListElement, DynamicQueryElement, DynamicQueryC
from pickle import loads, dumps
from joblib import Memory
import csv
import logging

In [2]:
df = pd.read_csv('/home/danylo/Desktop/Master/Data/reactions_clean.csv')
df.head()

,ID,ReactionSmiles,Yield,ReactionSmilesClean,Year
0,ord-e02e86c6722649c19d1c4d8e45a67838,Br[C:2]1[CH:9]=[CH:8][C:5]([C:6]#[N:7])=[CH:4]...,93.000000,N#CC1=CC=C(Br)C=C1.CCCCCC.[Li]CCCC.COB(OC)OC.C...,1994.0
1,ord-fbf2878a83214f49ba4bde529b07025e,[C:1]([C:3]1[CH:8]=[CH:7][C:6](B(O)O)=[CH:5][C...,30.000000,N#CC1=CC=C(B(O)O)C=C1.CCCCCCCSC1=CC=C(Br)C=C1....,1994.0
2,ord-6b762dd0417b4f4e8cfeb37ae62400fd,[CH2:1]([OH:10])[C:2]([CH2:4][O:5]P(O)(O)=O)=[...,30.000000,O=C(CO)COP(=O)(O)O.[N-]=[N+]=NC(C=O)CO.[OH-].[...,1994.0
3,ord-bbb9d39e511b426daffd167319c568a7,O1CCCC1.[CH3:6][C:7]([N:16]1[CH2:20][C:19]([CH...,87.599998,C1CCOC1.CC1(CBr)CN(C(C)(C)C2=CC(Cl)=CC=C2)C(=O...,1994.0
4,ord-f24b5aa0f9364497b2b28cd16abe3863,[CH3:1][C:2]([N:10]1[CH2:14][CH:13]([CH2:15]Br...,71.000000,CC(C)(C1=CC=CC=C1)N1CC(CBr)C(C2=CC(F)=CC=C2)C1...,1994.0


In [14]:
memory = Memory("cachedir", verbose=0)

def get_directory_size(directory):
    """Calculate the total size of all files in the given directory."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def manage_cache(memory, max_size_mb=500):
    """Check the size of the cache and clear it if it exceeds max_size_mb."""
    cache_dir = memory.location
    size_bytes = get_directory_size(cache_dir)
    size_mb = size_bytes / (1024 * 1024)  # convert bytes to megabytes

    if size_mb > max_size_mb:
        print(f"Cache size ({size_mb:.2f} MB) exceeds the limit of {max_size_mb} MB. Clearing cache.")
        memory.clear()

def split_list(data, num_chunks):
    chunk_size = len(data) // num_chunks
    chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
    if len(chunks) > num_chunks:
        last_chunk = chunks[-2] + chunks[-1]
        chunks[-2] = last_chunk
        chunks.pop()
    return chunks

def save_to_csv(data, filename="output_reaction_ID_2.csv"):
    """
    Saves specified data from a dictionary where each key has a list of tuples,
    and each tuple's second element (index 1) is saved to a CSV file.

    Args:
    data (dict): Dictionary containing lists of tuples.
    filename (str): The name of the CSV file to write the data to.
    """
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for key, tuples_list in data.items():
            # Extracting only the ID from each tuple
            row = [key] + [tpl[0] for tpl in tuples_list if len(tpl) > 1]
            csv_writer.writerow(row)


In [4]:
memory.clear()

[Memory(location=cachedir/joblib)]: Flushing completely the cache


# Fluor chemistry


In [4]:
data_reaction_from_all_sets = {
    "CO to CF2": [], 
    "Primary OH to F": [],
    "Secondary OH to F": [],
    "Tertiary OH to F": [],
    "Aromatic OH to F": [],
    "COOH to COOF": [], 
    "CSOC to CSCF": []
}
data_amide_coupling_and_C_C = {
"Amide bond formation": [],
"formylation": [],
"amination": [],
"C-C bond formation": []
}

In [5]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Assuming memory cache and other necessary imports and configurations are handled above
@memory.cache
def str_to_reaction(id_smiles_tuples, batch_size=200):
    container = []
    total = len(id_smiles_tuples)
    for i in range(0, total, batch_size):
        batch = id_smiles_tuples[i:i + batch_size]
        for id_smile in batch:
            reaction_id, str_smiles = id_smile
            reaction = AllChem.ReactionFromSmarts(str_smiles)
            container.append((reaction_id, reaction))   
        processed = min(i + batch_size, total)
        progress = (processed / total) * 100
        logging.info(f"Processed {processed}/{total} ({progress:.2f}%) reactions.")
    logging.info("Processing complete.")
    return container

@memory.cache
def filter_reactions_with_fluorine_in_products(id_reaction_tuples):
    # Check if fluorine is in any of the products
    # If fluorine is found, add the original reaction tuple (including ID) to the list
    # Return list with the reactions where fluorine is present in the products
    fluorine = Chem.MolFromSmiles('F')
    
    # List to hold the reactions where fluorine is present in the products
    reactions_with_fluorine = []
    
    for reaction_id, reaction in id_reaction_tuples:
        product_templates = [reaction.GetProductTemplate(i) for i in range(reaction.GetNumProductTemplates())]
        if any(prod.HasSubstructMatch(fluorine) for prod in product_templates if prod):
            reactions_with_fluorine.append((reaction_id, reaction))
    return reactions_with_fluorine

def rdk_to_CGR(id_reaction_tuples):
    # Process each reaction tuple to convert to SMILES and then to CGR
    smiles_mapped_data = []
    CGRs = []
    smiles_parser = SMILESRead.create_parser(ignore=True)
    for reaction_id, rxn in id_reaction_tuples:
        try:
            reaction = smiles_parser(rdkit.Chem.rdChemReactions.ReactionToSmiles(rxn))
            smiles_mapped_data.append((reaction_id, reaction))
        except Exception as e:
            print(f"Error processing reaction ID {reaction_id}: {e}")
            continue
    for reaction_id, reaction_data in smiles_mapped_data:
        try: # Check if reaction_data is properly parsed
                cgr = reaction_data.compose()
                CGRs.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error composing CGR for reaction ID {reaction_id}: {e}")
            continue
    return CGRs

 
def find_primary_OH_to_F(id_cgr_tuples):
    rule_primary = QueryCGRContainer()
    rule_primary.add_atom("O", 1)
    rule_primary.add_atom("C", 2, hybridization=1, p_hybridization=1, neighbors=2, p_neighbors=2)
    rule_primary.add_atom("F", 3)
    rule_primary.add_atom("C", 4)
    rule_primary.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
    rule_primary.add_bond(2, 3, DynamicBond(None, 1))
    rule_primary.add_bond(2, 4, 1)
    primary_Alcohol_to_Fluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule_primary:
                primary_Alcohol_to_Fluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return primary_Alcohol_to_Fluor
    
def find_secondary_OH_to_F(id_cgr_tuples):
    rule_secondary = QueryCGRContainer()
    rule_secondary.add_atom("O", 1)
    rule_secondary.add_atom("C", 2, hybridization=1, p_hybridization=1, neighbors=3, p_neighbors=3)
    rule_secondary.add_atom("F", 3)
    rule_secondary.add_atom("C", 4)
    rule_secondary.add_atom("C", 5)
    rule_secondary.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
    rule_secondary.add_bond(2, 3, DynamicBond(None, 1))
    rule_secondary.add_bond(2, 4, 1)
    rule_secondary.add_bond(2, 5, 1)
    secondary_Alcohol_to_Fluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule_secondary:
                secondary_Alcohol_to_Fluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return secondary_Alcohol_to_Fluor

def find_tertiary_OH_to_F(id_cgr_tuples):
    rule_tertiary = QueryCGRContainer()
    rule_tertiary.add_atom("O", 1)
    rule_tertiary.add_atom("C", 2, hybridization=1, p_hybridization=1)
    rule_tertiary.add_atom("F", 3)
    rule_tertiary.add_atom("C", 4)
    rule_tertiary.add_atom("C", 5)
    rule_tertiary.add_atom("C", 6)
    rule_tertiary.add_bond(1, 2, DynamicBond(1, None))  
    rule_tertiary.add_bond(2, 3, DynamicBond(None, 1))
    rule_tertiary.add_bond(2, 4, 1)
    rule_tertiary.add_bond(2, 5, 1)
    rule_tertiary.add_bond(2, 6, 1)
    tertiary_Alcohol_to_Fluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule_tertiary:
                tertiary_Alcohol_to_Fluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return tertiary_Alcohol_to_Fluor

def find_aromatic_OH_to_F (id_cgr_tuples):
    rule_aromatic = QueryCGRContainer()
    rule_aromatic.add_atom("O", 1)
    rule_aromatic.add_atom("C", 2, hybridization=4, p_hybridization=4)
    rule_aromatic.add_atom("F", 3)
    rule_aromatic.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
    rule_aromatic.add_bond(2, 3, DynamicBond(None, 1))
    Phenol_to_Fluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule_aromatic:
                Phenol_to_Fluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return Phenol_to_Fluor

def find_CO_to_CF2(id_cgr_tuples):
    rule2 = QueryCGRContainer()
    rule2.add_atom("O", 1)
    rule2.add_atom("C", 2, hybridization = 2, p_hybridization=1)
    rule2.add_atom("F", 3)
    rule2.add_atom("F", 4)
    rule2.add_bond(1, 2, DynamicBond(2, None))
    rule2.add_bond(2, 3, DynamicBond(None, 1))
    rule2.add_bond(2, 4, DynamicBond(None, 1))
    Ketone_to_Difluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule2:
                Ketone_to_Difluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return Ketone_to_Difluor

def find_COOH_to_COF(id_cgr_tuples):
    rule3 = QueryCGRContainer()
    rule3.add_atom("O", 1)
    rule3.add_atom("C", 2, hybridization = 2, p_hybridization=2)
    rule3.add_atom("O", 3)
    rule3.add_atom("C", 5)
    rule3.add_atom("F", 4)
    rule3.add_bond(2, 3, DynamicBond(1, None))
    rule3.add_bond(2, 5, 1)
    rule3.add_bond(1, 2, 2)
    rule3.add_bond(2, 4, DynamicBond(None, 1))
    Acyl_fluor = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule3:
                Acyl_fluor.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return Acyl_fluor

def find_CSOC_to_CSCF (id_cgr_tuples):
    rule4 = QueryCGRContainer()
    rule4.add_atom("O", 1)
    rule4.add_atom("S", 2, hybridization = 2, p_hybridization=1)
    rule4.add_atom("C", 3)
    rule4.add_atom("C", 4)
    rule4.add_atom("F", 5)
    rule4.add_bond(1, 2, DynamicBond(2, None))
    rule4.add_bond(2, 3, 1)
    rule4.add_bond(2, 4, 1)
    rule4.add_bond(4, 5, DynamicBond(None, 1))
    reactions = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule4:
                reactions.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return reactions

def find_COOH_CONHC (id_cgr_tuples):
    rule5 = QueryCGRContainer()
    rule5.add_atom("O", 1)
    rule5.add_atom("C", 2, hybridization = 2, p_hybridization=2)
    rule5.add_atom("O", 3)
    rule5.add_atom("C", 4)
    rule5.add_atom("N", 5)
    rule5.add_atom("C", 6)
    rule5.add_bond(2, 3, DynamicBond(1, None))
    rule5.add_bond(2, 5, DynamicBond(None, 1))
    rule5.add_bond(1, 2, 2)
    rule5.add_bond(4, 5,  1)
    rule5.add_bond(2, 6, 1)
    amide_formation = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule5:
                amide_formation.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return amide_formation

def C_C_find (id_cgr_tuples):
    rule6 = QueryCGRContainer()
    rule6.add_atom("C", 1)
    rule6.add_atom("C", 2)
    rule6.add_bond(1, 2, DynamicBond(None, 1))
    C_C_reaction = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule6:
                C_C_reaction.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return C_C_reaction

def find_amination (id_cgr_tuples):
    rule7 = QueryCGRContainer()
    rule7.add_atom("C", 1, hybridization = 1, p_hybridization=1)
    rule7.add_atom("N", 2)
    rule7.add_bond(1, 2, DynamicBond(None, 1))
    amination = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule7:
                amination.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return amination

def find_formylation (id_cgr_tuples):
    rule8 = QueryCGRContainer()
    rule8.add_atom("C", 1, hybridization = 1, p_hybridization=1)
    rule8.add_atom("C", 2, hybridization = 2, p_hybridization=2)
    rule8.add_atom("O", 3)
    rule8.add_atom("H", 4)
    rule8.add_bond(1, 2, DynamicBond(None, 1))
    rule8.add_bond(2, 3, 2)
    rule8.add_bond(2, 4, 1)
    formylation = []
    for reaction_id, cgr in id_cgr_tuples:
        try:
            if cgr > rule8:
                formylation.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error matching CGR for reaction ID {reaction_id}: {e}")
            continue
    return formylation

def append_data_to_sets(reaction_groups):
    labels = ["Primary OH to F", "Secondary OH to F", "Tertiary OH to F", "Aromatic OH to F", "CO to CF2", "COOH to COOF", "CSOC to CSCF"]
    for reactions, label in zip(reaction_groups, labels):
        for reaction_and_id in reactions:
            data_reaction_from_all_sets[label].append(reaction_and_id)

def append_data_to_sets2(reaction_groups):
    labels = ["Amide bond formation", "formylation", "amination", "C-C bond formation"]
    for reactions, label in zip(reaction_groups, labels):
        for reaction_and_id in reactions:
            data_amide_coupling_and_C_C[label].append(reaction_and_id)
            
def process_batch(chunk_num):
    manage_cache(memory, max_size_mb=100)
    logging.info("Converting strings to RDKit container")
    reactions = str_to_reaction(chunk_num, batch_size=200)
    logging.info("Filtering reactions by presence of Fluorine in product")
    filtered_reactions = filter_reactions_with_fluorine_in_products(reactions)
    logging.info("RDKit container converting to CGR")
    CGRs = rdk_to_CGR(filtered_reactions)
    logging.info("Starting finding reactions: Dexofluronation")
    primary_OH_to_F = find_primary_OH_to_F(CGRs)
    logging.info(f"Complete, total {len(primary_OH_to_F)} with primary alcohols")
    secondary_OH_to_F = find_secondary_OH_to_F(CGRs)
    logging.info(f"Complete, total {len(secondary_OH_to_F)} with secondary alcohols")
    tertiary_OH_to_F = find_tertiary_OH_to_F(CGRs)
    logging.info(f"Complete, total {len(tertiary_OH_to_F)} with tertiary alcohols")
    aromatic_OH_to_F = find_aromatic_OH_to_F(CGRs)
    logging.info(f"Complete, total {len(aromatic_OH_to_F)} with aromatic alcohols")
    CO_to_CF2 = find_CO_to_CF2(CGRs)
    logging.info(f"Complete, total {len(CO_to_CF2)} with Carbonyl")
    COOH_to_COF = find_COOH_to_COF(CGRs)
    logging.info(f"Complete, total {len(COOH_to_COF)} with Acid to Acyl Fluor")
    CSOC_to_CSCF = find_CSOC_to_CSCF(CGRs)
    logging.info(f"Complete, total {len(CSOC_to_CSCF)} with Sulfoxide")
    # Assuming data_reaction_from_all_sets is initialized
    append_data_to_sets([primary_OH_to_F, secondary_OH_to_F, tertiary_OH_to_F, aromatic_OH_to_F, CO_to_CF2, COOH_to_COF, CSOC_to_CSCF])
    manage_cache(memory, max_size_mb=100)
    logging.info("Success, proceed to next chunk of data")

@memory.cache
def process_batch2(chunk_num):
    smiles_mapped_data = []
    CGRs = []
    smiles_parser = SMILESRead.create_parser(ignore=True)
    for reaction_id, rxn in chunk_num:
        try:
            reaction = smiles_parser(rxn)
            smiles_mapped_data.append((reaction_id, reaction))
        except Exception as e:
            print(f"Error processing reaction ID {reaction_id}: {e}")
            continue
    for reaction_id, reaction_data in smiles_mapped_data:
        try: # Check if reaction_data is properly parsed
                cgr = reaction_data.compose()
                CGRs.append((reaction_id, cgr))
        except Exception as e:
            print(f"Error composing CGR for reaction ID {reaction_id}: {e}")
            continue
    logging.info("Starting finding reactions: Amide formation")
    amide_formation = find_COOH_CONHC(CGRs)
    logging.info(f"Complete, total {len(amide_formation)} with Amide formation")
    logging.info("Starting finding reactions: formylation")
    formyl_formation = find_formylation(CGRs)
    logging.info(f"Complete, total {len(formyl_formation)} with formylation")
    logging.info("Starting finding reactions: amination")
    amination = find_amination(CGRs)
    logging.info(f"Complete, total {len(amination)} with amination")
    C_C_formation = C_C_find(CGRs)
    logging.info(f"Complete, total {len(C_C_formation)} with C-C bond formation")
    append_data_to_sets2([amide_formation, formyl_formation, amination, C_C_formation])
    manage_cache(memory, max_size_mb=100)
    logging.info("Success, proceed to next chunk of data")


In [6]:
data = list(zip(df['ID'], df['ReactionSmiles']))
num_chunks = 10  # or any number that alows your memory capacity 
data_chunks = split_list(data, num_chunks)

In [7]:
process_batch2(data_chunks[0])

Error composing CGR for reaction ID ord-d46f8ae9899f4d65b382f4f00936da42: atoms with number {1} not equal
Error composing CGR for reaction ID ord-5d8ff1fa3347412d91402efd60a5118d: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-8a48eb41c3be4040a3335fc3d682c0e4: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-d37642a0b8804cfa8283f6420709ecc4: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-14ad2d0834f64d7f8f3ab66c60ba5885: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-f906fe57521d43c9aaebe51e0e7b2751: atoms with number {1} not equal
Error composing CGR for reaction ID ord-76fccefca238440694ba29e5f24d0730: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-869b3f634a7d4095af2099a2780f3f30: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-9e56941248144bd5a6a89aa82d073deb: 'dict' object has no att

2024-05-13 17:13:00,503 - INFO - Starting finding reactions: Amide formation


Error composing CGR for reaction ID ord-318e08feda5e4e6782a772d5aae8add1: atoms with number {7} not equal


2024-05-13 17:14:09,728 - INFO - Complete, total 3073 with Amide formation
2024-05-13 17:14:09,729 - INFO - Starting finding reactions: formylation
2024-05-13 17:14:15,662 - INFO - Complete, total 0 with formylation
2024-05-13 17:14:15,662 - INFO - Starting finding reactions: amination
2024-05-13 17:14:19,771 - INFO - Complete, total 6112 with amination
2024-05-13 17:14:28,477 - INFO - Complete, total 15603 with C-C bond formation
2024-05-13 17:14:28,480 - INFO - Success, proceed to next chunk of data


In [10]:
process_batch2(data_chunks[1])

Error composing CGR for reaction ID ord-cfa48b1f17e845dfb427b801578a75a5: atoms with number {11} not equal
Error composing CGR for reaction ID ord-9d753926ece44f7d9bf82447a162fea8: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-157622979b1144f08be1cc2d477957ad: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-32807e2eed70472e9849eca1a9ea38c3: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-bfc7f53bb4ee449eb152bc03f3b2994f: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-0054c2e06f194b4ca59f4d7e1911cb2f: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-426138b7bf014f07b1bc01684fe58d7d: atoms with number {11} not equal
Error composing CGR for reaction ID ord-3e2dff2775f04df88a4b1381300d88e3: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-410ffae9fa654d8ab0212ec147542f7a: atoms with number {9} 

2024-05-13 17:20:13,547 - INFO - Starting finding reactions: Amide formation
2024-05-13 17:21:20,473 - INFO - Complete, total 3398 with Amide formation
2024-05-13 17:21:20,474 - INFO - Starting finding reactions: formylation
2024-05-13 17:21:26,249 - INFO - Complete, total 0 with formylation
2024-05-13 17:21:26,249 - INFO - Starting finding reactions: amination
2024-05-13 17:21:30,432 - INFO - Complete, total 6049 with amination
2024-05-13 17:21:39,101 - INFO - Complete, total 14956 with C-C bond formation
2024-05-13 17:21:39,104 - INFO - Success, proceed to next chunk of data


In [11]:
process_batch2(data_chunks[2])

Error composing CGR for reaction ID ord-515947eac51f4236977fc35b3ddaaa6e: atoms with number {8} not equal
Error composing CGR for reaction ID ord-146a4bc958f94495b7633bbfbadc136e: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-d41089aefb8840d7900b79b66001bbf6: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-5da203cebbdb41e5baf92729ca7f8cee: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-c02c15bab89549f2a66521019a2194e4: atoms with number {13} not equal
Error composing CGR for reaction ID ord-d7daae02ffee47a89fb280c653ca915a: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-4b5f85785f8844309dea1e4b0cd826cc: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-4934603036504848996e948d8393bdb2: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-7811a123518b465ca55e17b0c789a52a: 'dict' object has no at

2024-05-13 17:26:18,336 - INFO - Starting finding reactions: Amide formation
2024-05-13 17:27:31,176 - INFO - Complete, total 2694 with Amide formation
2024-05-13 17:27:31,177 - INFO - Starting finding reactions: formylation
2024-05-13 17:27:37,029 - INFO - Complete, total 0 with formylation
2024-05-13 17:27:37,029 - INFO - Starting finding reactions: amination
2024-05-13 17:27:41,365 - INFO - Complete, total 6627 with amination
2024-05-13 17:27:50,332 - INFO - Complete, total 13685 with C-C bond formation
2024-05-13 17:27:50,335 - INFO - Success, proceed to next chunk of data


In [12]:
process_batch2(data_chunks[3])

Error composing CGR for reaction ID ord-3fbc40ca1453460db25905b51e16bba3: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-8ff0a030f282471dabd2c5ac7771f014: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-59744a70c00244609ffde29ee4d102df: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-e92c83b6fc5c41b6952ad2903c7a142b: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-c8e74421a820457f9bf41d6857d06f16: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-63b69af0c1174b4fba5ed1c0359d8589: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-b4764b344aac40128a11ce0aaf2b7c0f: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-b8a4803101984ccca32e68903f55759e: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-6becdaf5a0664db08c43ec074284ed98: 'dict'

2024-05-13 17:33:00,435 - INFO - Starting finding reactions: Amide formation
2024-05-13 17:34:08,017 - INFO - Complete, total 3615 with Amide formation
2024-05-13 17:34:08,017 - INFO - Starting finding reactions: formylation
2024-05-13 17:34:13,906 - INFO - Complete, total 0 with formylation
2024-05-13 17:34:13,906 - INFO - Starting finding reactions: amination
2024-05-13 17:34:18,060 - INFO - Complete, total 6042 with amination
2024-05-13 17:34:26,857 - INFO - Complete, total 14805 with C-C bond formation
2024-05-13 17:34:26,874 - WARNING - [Memory(location=cachedir/joblib)]: Flushing completely the cache
2024-05-13 17:34:26,889 - INFO - Success, proceed to next chunk of data


Cache size (117.99 MB) exceeds the limit of 100 MB. Clearing cache.


In [13]:
process_batch2(data_chunks[4])

Error composing CGR for reaction ID ord-4047409c32914fd89f5d465eaf16a581: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-c207a39eff0f416687bdcde0535fdae9: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-de6a7e728a8449c2aa7cab913f494e7c: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-88a3b9201dcd45e6b90874ddd541b6db: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-5d0f7acfcc8a42bf829315cc58648de9: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-d84b4293ccb4438b9ece6d4dd8bc2d5f: 'dict' object has no attribute 'compose'
Error composing CGR for reaction ID ord-a14ddb4c0c604f04984d4081fb02e9ff: atoms with number {4} not equal
Error composing CGR for reaction ID ord-6ea240dd4e12411caecdd28f674a88eb: atoms with number {18} not equal
Error composing CGR for reaction ID ord-457005cf08bb4c999d05636ff6094d92: atoms with number {1} n

2024-05-13 17:41:01,958 - INFO - Starting finding reactions: Amide formation
2024-05-13 17:42:28,106 - INFO - Complete, total 2999 with Amide formation
2024-05-13 17:42:28,107 - INFO - Starting finding reactions: formylation
2024-05-13 17:42:34,850 - INFO - Complete, total 0 with formylation
2024-05-13 17:42:34,851 - INFO - Starting finding reactions: amination
2024-05-13 17:42:39,730 - INFO - Complete, total 6125 with amination
2024-05-13 17:42:50,882 - INFO - Complete, total 14046 with C-C bond formation
2024-05-13 17:42:50,885 - INFO - Success, proceed to next chunk of data


In [15]:
save_to_csv(data_amide_coupling_and_C_C)

In [ ]:
process_batch2(data_chunks[5])

In [ ]:
process_batch2(data_chunks[6])

In [ ]:
process_batch2(data_chunks[7])

In [ ]:
process_batch2(data_chunks[8])

In [ ]:
process_batch2(data_chunks[9])

In [ ]:
len(data_reaction_from_all_sets['Primary OH to F'])

In [84]:
len(data_reaction_from_all_sets['Secondary OH to F'])

295

In [85]:
len(data_reaction_from_all_sets['Tertiary OH to F'])

191

In [86]:
len(data_reaction_from_all_sets['CO to CF2'])

10

In [87]:
len(data_reaction_from_all_sets["CSOC to CSCF"])

8

In [88]:
save_to_csv(data_reaction_from_all_sets)

In [63]:
    rule_secondary = QueryCGRContainer()
    rule_secondary.add_atom("O", 1)
    rule_secondary.add_atom("C", 2, hybridization=1, p_hybridization=1, neighbors=3, p_neighbors=3)
    rule_secondary.add_atom("F", 3)
    rule_secondary.add_atom("C", 4)
    rule_secondary.add_atom("C", 5)
    rule_secondary.add_atom("H", 6)
    rule_secondary.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
    rule_secondary.add_bond(2, 3, DynamicBond(None, 1))
    rule_secondary.add_bond(2, 4, 1)
    rule_secondary.add_bond(2, 5, 1)
    rule_secondary.add_bond(2, 6, 1)
    rule_secondary.clean2d()
    rule_secondary

In [35]:
    rule_tertiary = QueryCGRContainer()
    rule_tertiary.add_atom("O", 1)
    rule_tertiary.add_atom("C", 2, hybridization=1, p_hybridization=1)
    rule_tertiary.add_atom("F", 3)
    rule_tertiary.add_atom("C", 4)
    rule_tertiary.add_atom("C", 5)
    rule_tertiary.add_atom("C", 6)
    rule_tertiary.add_bond(1, 2, DynamicBond(1, None))  
    rule_tertiary.add_bond(2, 3, DynamicBond(None, 1))
    rule_tertiary.add_bond(2, 4, 1)
    rule_tertiary.add_bond(2, 5, 1)
    rule_tertiary.add_bond(2, 6, 1)
    rule_tertiary.clean2d()
    rule_tertiary

In [26]:
    rule2 = QueryCGRContainer()
    rule2.add_atom("O", 1)
    rule2.add_atom("C", 2, hybridization = 2, p_hybridization=1)
    rule2.add_atom("F", 3)
    rule2.add_atom("F", 4)
    rule2.add_bond(1, 2, DynamicBond(2, None))
    rule2.add_bond(2, 3, DynamicBond(None, 1))
    rule2.add_bond(2, 4, DynamicBond(None, 1))
    rule2.clean2d() 
    rule2

In [56]:
    rule_primary = QueryCGRContainer()
    rule_primary.add_atom("O", 1)
    rule_primary.add_atom("C", 2, hybridization=1, p_hybridization=1, neighbors=2, p_neighbors=2)
    rule_primary.add_atom("F", 3)
    rule_primary.add_atom("C", 4)
    rule_primary.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
    rule_primary.add_bond(2, 3, DynamicBond(None, 1))
    rule_primary.add_bond(2, 4, 1)
    rule_primary.clean2d()
    rule_primary


In [19]:
rule8 = QueryCGRContainer()
rule8.add_atom("C", 1, hybridization = 1, p_hybridization=1)
rule8.add_atom("C", 2, hybridization = 1, p_hybridization=1)
rule8.add_atom("O", 3)
rule8.add_atom("H", 4)
rule8.add_bond(1, 2, DynamicBond(None, 1))
rule8.add_bond(2, 3, 2)
rule8.add_bond(2, 4, 1)
rule8.clean2d()
rule8

In [18]:
rule5 = QueryCGRContainer()
rule5.add_atom("O", 1)
rule5.add_atom("C", 2, hybridization = 2, p_hybridization=2)
rule5.add_atom("O", 3)
rule5.add_atom("C", 4)
rule5.add_atom("N", 5)
rule5.add_atom("C", 6)
rule5.add_bond(2, 3, DynamicBond(1, None))
rule5.add_bond(2, 5, DynamicBond(None, 1))
rule5.add_bond(1, 2, 2)
rule5.add_bond(4, 5,  1)
rule5.add_bond(2, 6, 1)
rule5.clean2d()
rule5

In [4]:
smiles_parser = SMILESRead.create_parser(ignore=True)
reaction_test = "[OH:1][C:2](=[O:3])-[O:4].[NH2:5][C:6]>>[O:3]=[C:2]([OH:1])[N:5][C:6]"
cgr = reaction_test.compose()
print(cgr > rule5)

AttributeError: 'str' object has no attribute 'compose'

In [67]:
rule_aromatic = QueryCGRContainer()
rule_aromatic.add_atom("O", 1)
rule_aromatic.add_atom("C", 2, hybridization=4, p_hybridization=4)
rule_aromatic.add_atom("F", 3)
rule_aromatic.add_bond(1, 2, DynamicBond(1, None))  # DynamicBond parameters may need to be verified for correctness
rule_aromatic.add_bond(2, 3, DynamicBond(None, 1))
rule_aromatic.clean2d()
rule_aromatic

In [1]:
    rule5 = QueryCGRContainer()
    rule5.add_atom("O", 1)
    rule5.add_atom("C", 2, hybridization = 2, p_hybridization=2)
    rule5.add_atom("O", 3)
    rule5.add_atom("C", 4)
    rule5.add_atom("N", 5)
    rule5.add_atom("H", 6)
    rule5.add_bond(2, 3, DynamicBond(1, None))
    rule5.add_bond(2, 5, DynamicBond(None, 1))
    rule5.add_bond(1, 2, 2)
    rule5.add_bond(4, 5,  1)
    rule5.add_bond(5, 6, 1)

NameError: name 'QueryCGRContainer' is not defined

In [ ]:
add_atom(atom: Union[DynamicElement, Element, int, str], *args, p_charge: int = 0, p_is_radical: bool = False, **kwargs)
